In [2]:
from scipy.io import arff
import pandas as pd
import os


In [5]:

contents = []
types = []
labels = []
for root, dirs, files in os.walk("../data_sets/data/"):
    for name in files:
        with open(os.path.join(root,name),'r') as f:
            try:
                x = pd.DataFrame(arff.loadarff(f))
            except:
                pass

In [6]:
x

,0,1,2,3,4,5,6,7,8,9,...,7017,7018,7019,7020,7021,7022,7023,7024,7025,7026
0,"[0.20055, 0.37951, 0.39641, 2.0472, 32.351, 0....","[0.20912, 0.49988, 0.47225, 1.9447, 14.786, 0....","[0.24866, 0.69592, 0.26713, 1.5548, -1.1523, 0...","[0.081483, 0.30734, 0.45879, 2.4928, 51.952, 0...","[0.18732, 0.61323, 0.2296, 1.4063, -7.3128, 0....","[0.22822, 0.49794, 0.35969, 1.7502, -47.717, 0...","[0.11109, 0.64744, 0.28971, 1.4705, 2.5349, 0....","[0.53232, 0.027059, 0.70554, 53.954, 299.58, 0...","[0.00902, 0.63202, 0.053735, 1.1263, -37.842, ...","[0.12408, 0.83837, 0.14204, 1.1694, -91.883, 0...",...,"[0.015903, 0.73014, 0.083767, 1.2865, -19.807,...","[-0.051896, 0.5363, 0.054014, 1.1061, -9.7422,...","[-0.031617, 0.81175, -0.2023, 0.61087, -44.011...","[0.025664, 0.76204, 0.09122, 1.1263, -62.155, ...","[0.069049, 0.68232, 0.069829, 1.1271, -54.533,...","[0.018371, 0.4741, -0.13619, 0.60839, -18.449,...","[-0.013359, 0.58354, -0.02265, 0.92896, -42.23...","[0.006338, 0.50276, 0.43923, 1.8736, 9.7417, 0...","[-0.041643, 0.8481, -0.12852, 0.57485, -121.92...","[0.014946, 0.94648, 0.03211, 1.0363, -20.581, ..."
1,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,None,None,None,None,None,None,None,None,None,None
